In [1]:
import os
import sys
import json
from tqdm import tqdm_notebook as tqdm
from pprint import pprint

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

## data load

In [2]:
ls ../mnt/inputs/origin/

data-science-bowl-2019.zip  specs.csv  train.csv     train_labels.csv
sample_submission.csv       test.csv   train.pkl.gz


In [3]:
trn_df = pd.read_pickle('../mnt/inputs/origin/train.pkl.gz')
# trn_df = pd.read_csv('../mnt/inputs/origin/train.csv')
trn_labels_df = pd.read_csv('../mnt/inputs/origin/train_labels.csv')
specs_df = pd.read_csv('../mnt/inputs/origin/specs.csv')
tst_df = pd.read_csv('../mnt/inputs/origin/test.csv')
sub_df = pd.read_csv('../mnt/inputs/origin/sample_submission.csv')

In [4]:
display(trn_df.shape, trn_df.head())

(11341042, 11)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [10]:
display(trn_labels_df.shape, trn_labels_df.head())

(17690, 7)

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [6]:
display(tst_df.shape, tst_df.head())

(1156414, 11)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [7]:
display(sub_df.shape, sub_df.head())

(1000, 2)

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3


## EDA

In [4]:
trn_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [10]:
ass_dt = pd.to_datetime(trn_df.timestamp)

In [11]:
dir(ass_dt.dt)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_add_delegate_accessors',
 '_constructor',
 '_delegate_method',
 '_delegate_property_get',
 '_delegate_property_set',
 '_deprecations',
 '_dir_additions',
 '_dir_deletions',
 '_freeze',
 '_get_values',
 '_reset_cache',
 'ceil',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'floor',
 'freq',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'microsecond',
 'minute',
 'month',
 'month_name',
 'nanosecond',
 'normalize',
 'quarter',
 'round',
 'second',
 'strftime',
 'time',
 '

In [12]:
ass_dt.dt.

0           30
1           30
2           30
3           30
4           30
            ..
11341037    31
11341038    31
11341039    31
11341040    31
11341041    31
Name: timestamp, Length: 11341042, dtype: int64

In [7]:
df = pd.read_pickle('../mnt/inputs/features/train/Worldcount.pkl')

In [9]:
df.set_index(['game_session', 'installation_id']).add_prefix('temp_').reset_index().head()

,game_session,installation_id,temp_title_type_cnt_12 Monkeys_Clip,temp_title_type_cnt_Air Show_Game,temp_title_type_cnt_All Star Sorting_Game,temp_title_type_cnt_Balancing Act_Clip,temp_title_type_cnt_Bird Measurer (Assessment)_Assessment,temp_title_type_cnt_Bottle Filler (Activity)_Activity,temp_title_type_cnt_Bubble Bath_Game,temp_title_type_cnt_Bug Measurer (Activity)_Activity,...,temp_wrd_type_cnt_CRYSTALCAVES_Game,temp_wrd_type_cnt_MAGMAPEAK_Activity,temp_wrd_type_cnt_MAGMAPEAK_Assessment,temp_wrd_type_cnt_MAGMAPEAK_Clip,temp_wrd_type_cnt_MAGMAPEAK_Game,temp_wrd_type_cnt_NONE_Clip,temp_wrd_type_cnt_TREETOPCITY_Activity,temp_wrd_type_cnt_TREETOPCITY_Assessment,temp_wrd_type_cnt_TREETOPCITY_Clip,temp_wrd_type_cnt_TREETOPCITY_Game
0,901acc108f55a5a1,0006a69f,1,0,153,0,0,0,0,0,...,0,102,0,2,131,2,250,0,7,153
1,77b8ee947eb84b4e,0006a69f,1,180,153,0,0,0,0,204,...,0,102,0,2,131,2,454,48,10,394
2,6bdf9623adc94d89,0006a69f,1,180,153,0,87,0,0,204,...,0,102,0,2,131,2,454,135,10,394
3,e7e7db2a241eadcc,0006a69f,2,180,203,0,87,110,115,204,...,0,322,0,5,401,3,781,170,16,444
4,9501794defd84e4d,0006a69f,2,180,203,0,87,110,115,204,...,0,322,0,5,401,3,781,187,16,444
